In [ ]:
#test
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

intact_dir = "/home/jovyan/test_sha/RTC_0514/intact"
tear_dir = "/home/jovyan/test_sha/RTC_0514/tear"

num_train_per_class = 205
num_test_intact = 30
num_test_tear = 107
num_folds = 5
batch_size = 64
epochs = 100
img_height, img_width, channels = 224, 224, 3

def load_and_preprocess_image(image_path):
    image = load_img(image_path, target_size=(img_height, img_width))
    image = img_to_array(image)
    image = image / 255.0  
    return image

intact_images_paths = [os.path.join(intact_dir, img) for img in os.listdir(intact_dir)]
tear_images_paths = [os.path.join(tear_dir, img) for img in os.listdir(tear_dir)]

random.seed(42)
intact_train_paths = random.sample(intact_images_paths, num_train_per_class)
tear_train_paths = random.sample(tear_images_paths, num_train_per_class)

intact_test_paths = random.sample(intact_images_paths, num_test_intact)
tear_test_paths = random.sample(tear_images_paths, num_test_tear)

intact_train_images = [load_and_preprocess_image(img) for img in intact_train_paths]
tear_train_images = [load_and_preprocess_image(img) for img in tear_train_paths]
intact_test_images = [load_and_preprocess_image(img) for img in intact_test_paths]
tear_test_images = [load_and_preprocess_image(img) for img in tear_test_paths]

intact_train_labels = [0] * len(intact_train_images)
tear_train_labels = [1] * len(tear_train_images)
intact_test_labels = [0] * len(intact_test_images)
tear_test_labels = [1] * len(tear_test_images)

train_images = np.array(intact_train_images + tear_train_images)
train_labels = np.array(intact_train_labels + tear_train_labels)
test_images = np.array(intact_test_images + tear_test_images)
test_labels = np.array(intact_test_labels + tear_test_labels)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, channels))
for layer in base_model.layers:
    layer.trainable = False

model_vgg = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_vgg.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_accuracy', target_accuracy=0.99, consecutive_epochs=5):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.target_accuracy = target_accuracy
        self.consecutive_epochs = consecutive_epochs
        self.count = 0

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)
        if (current_accuracy is not None) and (current_accuracy >= self.target_accuracy):
            self.count += 1
        else:
            self.count = 0

        if self.count >= self.consecutive_epochs:
            print(f"\nReached target accuracy of {self.target_accuracy} for {self.consecutive_epochs} consecutive epochs. Stopping training.")
            self.model.stop_training = True

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00001)
custom_callback = CustomCallback()

datagen = ImageDataGenerator()
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
test_generator = datagen.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)

kf = KFold(n_splits=num_folds, shuffle=True)
fold = 0
accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

for train_index, test_index in kf.split(train_images):
    fold += 1
    print(f"\nFold {fold}/{num_folds}")

    X_train, X_val = train_images[train_index], train_images[test_index]
    y_train, y_val = train_labels[train_index], train_labels[test_index]

    train_datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

    history = model_vgg.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[early_stop, custom_callback],
        verbose=0
    )

    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    
    y_pred = model_vgg.predict(val_generator)
    y_pred = np.round(y_pred).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)


print(f"Fold {fold} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")


print("\nCross Validation Results:")
print(f"Mean Accuracy: {np.mean(accuracy_scores)} ± {np.std(accuracy_scores)}")
print(f"Mean Precision: {np.mean(precision_scores)} ± {np.std(precision_scores)}")
print(f"Mean Recall: {np.mean(recall_scores)} ± {np.std(recall_scores)}")
print(f"Mean F1 Score: {np.mean(f1_scores)} ± {np.std(f1_scores)}")

history = model_vgg.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stop, custom_callback]
)

y_true = test_generator.y
y_pred = model_vgg.predict(test_generator)
y_pred = np.round(y_pred).astype(int)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

cm = confusion_matrix(y_true, y_pred)

print("\nTest Results:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

save_dir = '/home/jovyan/test_sha/RTC_0514/data/04'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, 'model.h5')
model_vgg.save(model_path)
print(f"Model saved to {model_path}")


In [ ]:
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

intact_dir = "/home/jovyan/test_sha/RTC_0514/intact"
tear_dir = "/home/jovyan/test_sha/RTC_0514/tear"

num_train_per_class = 205
num_test_intact = 30
num_test_tear = 107
num_folds = 5
batch_size = 64
epochs = 100
img_height, img_width, channels = 224, 224, 3

def load_and_preprocess_image(image_path):
    image = load_img(image_path, target_size=(img_height, img_width))
    image = img_to_array(image)
    image = image / 255.0  
    return image

intact_images_paths = [os.path.join(intact_dir, img) for img in os.listdir(intact_dir)]
tear_images_paths = [os.path.join(tear_dir, img) for img in os.listdir(tear_dir)]

random.seed(42)
intact_train_paths = random.sample(intact_images_paths, num_train_per_class)
tear_train_paths = random.sample(tear_images_paths, num_train_per_class)

intact_test_paths = random.sample(intact_images_paths, num_test_intact)
tear_test_paths = random.sample(tear_images_paths, num_test_tear)

intact_train_images = [load_and_preprocess_image(img) for img in intact_train_paths]
tear_train_images = [load_and_preprocess_image(img) for img in tear_train_paths]
intact_test_images = [load_and_preprocess_image(img) for img in intact_test_paths]
tear_test_images = [load_and_preprocess_image(img) for img in tear_test_paths]

intact_train_labels = [0] * len(intact_train_images)
tear_train_labels = [1] * len(tear_train_images)
intact_test_labels = [0] * len(intact_test_images)
tear_test_labels = [1] * len(tear_test_images)

train_images = np.array(intact_train_images + tear_train_images)
train_labels = np.array(intact_train_labels + tear_train_labels)
test_images = np.array(intact_test_images + tear_test_images)
test_labels = np.array(intact_test_labels + tear_test_labels)

# 建立VGG模型
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, channels))
for layer in base_model.layers:
    layer.trainable = False

model_vgg = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_vgg.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_accuracy', target_accuracy=0.99, consecutive_epochs=5):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.target_accuracy = target_accuracy
        self.consecutive_epochs = consecutive_epochs
        self.count = 0

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)
        if (current_accuracy is not None) and (current_accuracy >= self.target_accuracy):
            self.count += 1
        else:
            self.count = 0

        if self.count >= self.consecutive_epochs:
            print(f"\nReached target accuracy of {self.target_accuracy} for {self.consecutive_epochs} consecutive epochs. Stopping training.")
            self.model.stop_training = True

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00001)
custom_callback = CustomCallback()

datagen = ImageDataGenerator()
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
test_generator = datagen.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)

kf = KFold(n_splits=num_folds, shuffle=True)
fold = 0
accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

for train_index, test_index in kf.split(train_images):
    fold += 1
    print(f"\nFold {fold}/{num_folds}")

    X_train, X_val = train_images[train_index], train_images[test_index]
    y_train, y_val = train_labels[train_index], train_labels[test_index]

    train_datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

    history = model_vgg.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[early_stop, custom_callback],
        verbose=0
    )

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    y_pred = model_vgg.predict(val_generator)
    y_pred = np.round(y_pred).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

print(f"Fold {fold} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

print("\nCross Validation Results:")
print(f"Mean Accuracy: {np.mean(accuracy_scores)} ± {np.std(accuracy_scores)}")
print(f"Mean Precision: {np.mean(precision_scores)} ± {np.std(precision_scores)}")
print(f"Mean Recall: {np.mean(recall_scores)} ± {np.std(recall_scores)}")
print(f"Mean F1 Score: {np.mean(f1_scores)} ± {np.std(f1_scores)}")

history = model_vgg.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stop, custom_callback]
)

y_true = test_generator.y
y_pred = model_vgg.predict(test_generator)
y_pred = np.round(y_pred).astype(int)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

cm = confusion_matrix(y_true, y_pred)

print("\nTest Results:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

save_dir = '/home/jovyan/test_sha/RTC_0514/data/04'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, 'model.h5')
model_vgg.save(model_path)
print(f"Model saved to {model_path}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

tp = 24
fp = 6
fn = 20
tn = 87

conf_matrix = np.array([[tn, fp],
                        [fn, tp]])

labels = ['tear', 'intact']

df_cm = pd.DataFrame(conf_matrix, index=labels, columns=labels)

plt.figure(figsize=(6, 4))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
            xticklabels=labels, yticklabels=labels, annot_kws={"size": 15})
plt.ylabel('Actual', fontsize=10)
plt.xlabel('Predicted', fontsize=10)
plt.title('Confusion Matrix_Fold 4', fontsize=12)
plt.show()


In [ ]:
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

intact_dir = "/home/jovyan/test_sha/RTC_0514/intact"
tear_dir = "/home/jovyan/test_sha/RTC_0514/tear"

num_train_per_class = 205
num_test_intact = 30
num_test_tear = 107
num_folds = 5
batch_size = 64
epochs = 100
img_height, img_width, channels = 224, 224, 3

def load_and_preprocess_image(image_path):
    image = load_img(image_path, target_size=(img_height, img_width))
    image = img_to_array(image)
    image = image / 255.0  
    return image

intact_images_paths = [os.path.join(intact_dir, img) for img in os.listdir(intact_dir)]
tear_images_paths = [os.path.join(tear_dir, img) for img in os.listdir(tear_dir)]

random.seed(42)
intact_train_paths = random.sample(intact_images_paths, num_train_per_class)
tear_train_paths = random.sample(tear_images_paths, num_train_per_class)

intact_test_paths = random.sample(intact_images_paths, num_test_intact)
tear_test_paths = random.sample(tear_images_paths, num_test_tear)

intact_train_images = [load_and_preprocess_image(img) for img in intact_train_paths]
tear_train_images = [load_and_preprocess_image(img) for img in tear_train_paths]
intact_test_images = [load_and_preprocess_image(img) for img in intact_test_paths]
tear_test_images = [load_and_preprocess_image(img) for img in tear_test_paths]

intact_train_labels = [0] * len(intact_train_images)
tear_train_labels = [1] * len(tear_train_images)
intact_test_labels = [0] * len(intact_test_images)
tear_test_labels = [1] * len(tear_test_images)

train_images = np.array(intact_train_images + tear_train_images)
train_labels = np.array(intact_train_labels + tear_train_labels)
test_images = np.array(intact_test_images + tear_test_images)
test_labels = np.array(intact_test_labels + tear_test_labels)

# 建立VGG模型
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, channels))
for layer in base_model.layers:
    layer.trainable = False

model_vgg = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_vgg.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])


class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_accuracy', target_accuracy=0.99, consecutive_epochs=5):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.target_accuracy = target_accuracy
        self.consecutive_epochs = consecutive_epochs
        self.count = 0

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)
        if (current_accuracy is not None) and (current_accuracy >= self.target_accuracy):
            self.count += 1
        else:
            self.count = 0

        if self.count >= self.consecutive_epochs:
            print(f"\nReached target accuracy of {self.target_accuracy} for {self.consecutive_epochs} consecutive epochs. Stopping training.")
            self.model.stop_training = True

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00001)
custom_callback = CustomCallback()

datagen = ImageDataGenerator()
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
test_generator = datagen.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)

kf = KFold(n_splits=num_folds, shuffle=True)
fold = 0
accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

for train_index, test_index in kf.split(train_images):
    fold += 1
    print(f"\nFold {fold}/{num_folds}")

    X_train, X_val = train_images[train_index], train_images[test_index]
    y_train, y_val = train_labels[train_index], train_labels[test_index]

    train_datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

    history = model_vgg.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[early_stop, custom_callback],
        verbose=0
    )

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    y_pred = model_vgg.predict(val_generator)
    y_pred = np.round(y_pred).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    print(f"Fold {fold} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
mean_precision = np.mean(precision_scores)
std_precision = np.std(precision_scores)
mean_recall = np.mean(recall_scores)
std_recall = np.std(recall_scores)
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)

print("\nCross Validation Results:")
print(f"Mean Accuracy: {mean_accuracy} ± {std_accuracy}")
print(f"Mean Precision: {mean_precision} ± {std_precision}")
print(f"Mean Recall: {mean_recall} ± {std_recall}")
print(f"Mean F1 Score: {mean_f1} ± {std_f1}")

history = model_vgg.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stop, custom_callback]
)

y_true = test_generator.y
y_pred = model_vgg.predict(test_generator)
y_pred = np.round(y_pred).astype(int)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

cm = confusion_matrix(y_true, y_pred)

print("\nTest Results:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

save_dir = '/home/jovyan/test_sha/RTC_0514/data/04'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, 'model.h5')
model_vgg.save(model_path)
print(f"Model saved to {model_path}")